In [1]:
import os
import re
import requests
import sys
import json
from bs4 import BeautifulSoup
import ddddocr
import pandas as pd
import numpy as np
import six
import csv

In [2]:
# 宣告OCR物件
ocr = ddddocr.DdddOcr(show_ad=False)

# 重複執行直到驗證通過
while True:
    # 宣告Session物件
    session = requests.Session()
    # 發出http請求
    resp = session.get('https://bsr.twse.com.tw/bshtm/bsMenu.aspx', verify=False)

    # 如果回應200代表請求成功
    if resp.status_code == 200:
        # 使用 BeautifulSoup 解析 html
        soup = BeautifulSoup(resp.text, 'lxml')
        # 取得所有 'input' 物件
        nodes = soup.select('input')
        # 將欲傳送的資料存至 dictionary 當中
        params = {}
        for node in nodes:
            name = node.attrs['name']

            if name in ('RadioButton_Excd', 'Button_Reset'):
                continue

            if 'value' in node.attrs:
                params[node.attrs['name']] = node.attrs['value']
            else:
                params[node.attrs['name']] = ''

        # 取得圖片物件
        captcha_image = soup.select('#Panel_bshtm img')[0]['src']
        m = re.search(r'guid=(.+)', captcha_image)
        if m is None:
            exit(1)

        # 圖片存檔
        imgpath = '%s.jpg' % m.group(1)
        url = 'https://bsr.twse.com.tw/bshtm/' + captcha_image
        resp = requests.get(url, verify=False)
        if resp.status_code == 200:
            with open(imgpath, 'wb') as f:
                f.write(resp.content)
            os.system('open ' + imgpath)
            f.close()

        with open(imgpath, 'rb') as f:
            img_bytes = f.read()
        
        #利用OCR辨識圖片
        vcode = ocr.classification(img_bytes)
        print(vcode)
        f.close()

        # 圖片刪除
        try:
            os.remove(imgpath)
        except:
            pass
        params['CaptchaControl1'] = vcode

        #股票代碼
        params['TextBox_Stkno'] = '2317'
        print(json.dumps(params, indent=2))

        #送出資料
        resp = session.post('https://bsr.twse.com.tw/bshtm/bsMenu.aspx', data=params)
        if resp.status_code != 200:
            print('任務失敗: %d' % resp.status_code)
            continue

        #利用BeautifulSoup解析HTML
        soup = BeautifulSoup(resp.text, 'lxml')
        nodes = soup.select('#HyperLink_DownloadCSV')
        if len(nodes) == 0:
            print('任務失敗，沒有下載連結')
            continue

        #下載卷商分點進出CSV
        resp = session.get('https://bsr.twse.com.tw/bshtm/bsContent.aspx')
        with open('output.csv', 'w', encoding="utf-8-sig") as f:  #加-sig可以解決亂碼問題
            writer = csv.writer(f)
            for resp_line in resp.text.split('\n')[2:]:
                a = resp_line.split(',')
                print(a)
                if len(a) >= 4:
                    a[1] = re.sub(r'[0-9A-Za-z ]+', '', a[1])
                    if len(a) >= 10:
                        a[7] = re.sub(r'[0-9A-Za-z ]+', '', a[7])
                    writer.writerow(a)
            f.close()

        if resp.status_code != 200:
            print('任務失敗，無法下載卷商分點進出 CSV')
            continue
        break

U3DtV
{
  "__VIEWSTATE": "/wEPDwUJOTAxMjkxMjg3D2QWAgIDD2QWBAIBD2QWAmYPZBYEZg9kFgJmDxAPFgYeB0NoZWNrZWRnHglGb3JlQ29sb3IKAB4EXyFTQgIEZGRkZAIBD2QWAgIBDw8WBB8BCgAfAgIEZGQCAw9kFgQCAw8PFgIeBFRleHRlZGQCCQ8PFgIeB1Zpc2libGVoZGQYAgUeX19Db250cm9sc1JlcXVpcmVQb3N0QmFja0tleV9fFgMFElJhZGlvQnV0dG9uX05vcm1hbAUQUmFkaW9CdXR0b25fRXhjZAUQUmFkaW9CdXR0b25fRXhjZAUPQ2FwdGNoYUNvbnRyb2wxDwUkMDdlNTVlZWItZjFlNS00NjA3LTlmYTEtYjMwMmY2OTZhNDIxZFbSGgSIxU2utXHTp6FQjOc=",
  "__VIEWSTATEGENERATOR": "AA1F01CB",
  "__EVENTVALIDATION": "/wEdAAYlKQDr84CIGpldUuCk1KD6fMmuxAJNAJcNkRRsVeJwwqKURZav/+YrVMqaWE2hvMxJwK4Ohf3nRgXHkFhQen1PRSciNdPYWmINCrip1wqw01PJCA0uL9aE2sjICZqv6Gpc3RXiYKFLYIZR5D7T1jyG",
  "RadioButton_Normal": "RadioButton_Normal",
  "TextBox_Stkno": "2317",
  "CaptchaControl1": "U3DtV",
  "btnOK": "\u67e5\u8a62"
}
任務失敗，沒有下載連結
LXYR
{
  "__VIEWSTATE": "/wEPDwUJOTAxMjkxMjg3D2QWAgIDD2QWBAIBD2QWAmYPZBYEZg9kFgJmDxAPFgYeB0NoZWNrZWRnHglGb3JlQ29sb3IKAB4EXyFTQgIEZGRkZAIBD2QWAgIBDw8WBB8BCgAfAgIEZGQCAw9kFgQCAw8PFgIeBFRleHRlZGQCCQ

In [5]:
# 兩欄合併成一欄
data = pd.read_csv("D:/111第1學期/大數據/HW2/output.csv")
combine_df = pd.DataFrame()
# 取出分點資料表的兩欄資料，並合併成一欄
df_left = data.drop(['序號','序號.1', '券商.1', '價格.1', '買進股數.1', '賣出股數\r', 'Unnamed: 5'], axis=1)
df_right = data.drop(['序號', '序號.1', '券商', '價格', '買進股數', '賣出股數', 'Unnamed: 5'], axis=1)
df_right = df_right.rename(columns={'券商.1': '券商', '價格.1': '價格', '買進股數.1': '買進股數', '賣出股數\r': '賣出股數'})
combine_df = pd.concat([df_left, df_right], axis=0)
combine_df = combine_df.sort_values(["券商"], ascending=True, ignore_index=True)
combine_df = combine_df.dropna(axis=0, how='any')
combine_df['價格'] = combine_df['價格'].astype(float)
combine_df['買進股數'] = combine_df['買進股數'].astype(float)
combine_df['賣出股數'] = combine_df['賣出股數'].astype(float)
print(combine_df)

        券商     價格     買進股數     賣出股數
0       ?亞  105.0   1000.0   1000.0
1     ?亞網路  104.5      0.0   1000.0
2     ?亞網路  105.5      5.0      0.0
3     ?亞網路  105.0      0.0   1000.0
4     ?亞鑫豐  105.5    300.0   1000.0
...    ...    ...      ...      ...
1564  高橋中壢  104.5      0.0   1000.0
1565  高橋內壢  105.5   1000.0      0.0
1566  高橋內壢  105.0    100.0      0.0
1567  高橋龍潭  105.0  39000.0  36200.0
1568  高橋龍潭  104.5      0.0   1000.0

[1569 rows x 4 columns]


In [42]:
# 針對買進、賣出、買超、均買價、均賣價進行計算
temp = pd.DataFrame()
# 計算買進張數，每張為買進股數之千分之一
temp["買進"] = combine_df.groupby(by='券商').apply(lambda x: (x['買進股數']/1000).sum())
# 計算賣出張數，每張為賣出股數之千分之一
temp["賣出"] = combine_df.groupby(by='券商').apply(lambda x: (x['賣出股數']/1000).sum())
# 計算買超張數，買超為(買進股數-賣出股數)之千分之一
temp["買超"] = combine_df.groupby(by='券商').apply(lambda x: ((x['買進股數']-x['賣出股數'])/1000).sum())
# 計算均買價，均買價為當日買進總價除以總股數  #買進總股數
temp["均買價"] = combine_df.groupby(by='券商').apply(lambda x: ((x['價格']*x['買進股數'])/x['買進股數'].sum()).sum())
# 計算均賣價，均賣價為當日賣出總價除以總股數  #賣出總股數
temp["均賣價"] = combine_df.groupby(by='券商').apply(lambda x: ((x['價格']*x['賣出股數'])/x['賣出股數'].sum()).sum())


# 依照買超排序，並取出兩表(買超為正和買超為負的兩表)
temp = temp.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')
# 利用filter取出買超為正的row
positive = temp[temp['買超'] >= 0]
positive = positive.sort_values(by='買超', ascending=False).reset_index()
# 利用filter取出買超為負的row
negative = temp[temp['買超'] < 0]
negative = negative.sort_values(by='買超').reset_index()
del positive['index']
del negative['index']

# 將兩表合併，並儲存至csv
result = pd.concat([positive, negative], axis=1).round(2)
# result.set_axis(['券商', '買進', '賣出', '買超', '均買價', '均賣價']*2, axis=1, inplace=True)
print(result)
result.to_csv("result.csv",  encoding='utf-8-sig')

       券商       買進      賣出       買超     均買價     均賣價    券商       買進      賣出  \
0    台灣摩根  1525.00  464.00  1061.00  104.99  104.68  台灣匯立   955.32  3108.0   
1    瑞士信貸   856.00   92.00   764.00  105.05  105.00    美林  1921.00  3380.0   
2    港商法興   446.00    6.00   440.00  105.00  105.00  美商高盛   731.00  1894.4   
3    華南南京   414.06   12.11   401.95  104.99  105.04  凱基台北   474.00  1123.0   
4      統一   423.60   33.10   390.50  105.01  104.87    富邦    64.00   614.0   
..    ...      ...     ...      ...     ...     ...   ...      ...     ...   
462  國票中正     2.00    2.00     0.00  104.75  105.50   NaN      NaN     NaN   
463  土　　銀     2.00    2.00     0.00  104.75  105.00   NaN      NaN     NaN   
464  國票中港     1.00    1.00     0.00  105.00  104.50   NaN      NaN     NaN   
465  國票敦北    54.00   54.00     0.00  105.00  105.00   NaN      NaN     NaN   
466    ?亞     1.00    1.00     0.00  105.00  105.00   NaN      NaN     NaN   

          買超     均買價     均賣價  
0   -2152.68  104.92  104.86  
1